# 1. Script details

Name of script: OpenDataAPIQuery <br>
Description:  Using Python to query the NHSBSA open data portal API. <br>
Created by: Ryan Leggett (NHSBSA) <br>
Created on: 26-06-2022 <br>
Python version: created in 3.8

# 2. Load packages

List packages we will use

In [ ]:
import grequests
import pandas as pd
import re
import requests
import warnings
import urllib.parse

warnings.simplefilter("ignore", category=UserWarning)

Install packages if they aren't already using `Pip/Conda install -r requirements.txt`

# 3. Define variablesDefine the url for the API call

Define the url for the API call

In [ ]:
base_endpoint = 'https://opendata.nhsbsa.net/api/3/action/'
package_list_method = 'package_list'     # List of data-sets in the portal
package_show_method = 'package_show?id=' # List all resources of a data-set
action_method = 'datastore_search_sql?'  # SQL action method

Send API call to get list of data-sets

In [ ]:
datasets_response = requests.get(base_endpoint +  package_list_method).json()

Now lets have a look at the data-sets currently available

In [ ]:
print(datasets_response['result'])

For this example we're interested in the English Prescribing Dataset (EPD).
We know the name of this data-set so can set this manually, or access it 
from datasets_response.

In [ ]:
dataset_id = "english-prescribing-data-epd"

# 4. API calls for single month

Define the parameters for the SQL query

In [ ]:
resource_name = 'EPD_202001' # For EPD resources are named EPD_YYYYMM
pco_code = '13T00' # Newcastle Gateshead CCG
bnf_chemical_substance = '0407010H0' # Paracetamol

Build SQL query (WHERE criteria should be enclosed in single quotes)

In [ ]:
single_month_query = "SELECT * " \
                     f"FROM `{resource_name}` " \
                     f"WHERE pco_code = '{pco_code}' " \
                     f"AND bnf_chemical_substance = '{bnf_chemical_substance}'"

Build API call

In [ ]:
single_month_api_call = f"{base_endpoint}" \
                        f"{action_method}" \
                        "resource_id=" \
                        f"{resource_name}" \
                        "&" \
                        "sql=" \
                        f"{urllib.parse.quote(single_month_query)}" # Encode spaces in the url

Grab the response JSON as a list

In [ ]:
single_month_response = requests.get(single_month_api_call).json()

Extract records in the response to a dataframe

In [ ]:
single_month_df  = pd.json_normalize(single_month_response['result']['result']['records'])

Lets have a quick look at the data

In [ ]:
single_month_df.head()

You can use any of the fields listed in the data-set within the SQL query as 
part of the select or in the where clause in order to filter.

Information on the fields present in a data-set and an accompanying data 
dictionary can be found on the page for the relevant data-set on the Open Data 
Portal.

# 5. API calls for data for multiple months

Now that you have extracted data for a single month, you may want to get the 
data for several months, or a whole year.

Firstly we need to get a list of all of the names and resource IDs for every 
EPD file. We therefore extract the metadata for the EPD dataset.

In [ ]:
metadata_repsonse  = requests.get(f"{base_endpoint}" \
                                  f"{package_show_method}" \
                                  f"{dataset_id}").json()

Resource names and IDs are kept within the resources table returned from the 
package_show_method call.

In [ ]:
resources_table  = pd.json_normalize(metadata_repsonse['result']['resources'])

We only want data for one calendar year, to do this we need to look at the 
name of the data-set to identify the year. For this example we're looking at 
2020.

In [ ]:
resource_name_list = resources_table[resources_table['name'].str.contains('2020')]['name']

## 5.1. For loop

We can do this with a for loop that makes all of the individual API calls for 
you and combines the data together into one dataframe

Initialise dataframe that data will be saved to

In [ ]:
for_loop_df = pd.DataFrame()

As each individual month of EPD data is so large it will be unlikely that your 
local system will have enough RAM to hold a full year's worth of data in 
memory. Therefore we will only look at a single CCG and chemical substance as 
we did previously

Loop through resource_name_list and make call to API to extract data, then 
bind each month together to make a single data-set

In [ ]:
for month in resource_name_list:
    
    # Build temporary SQL query
    tmp_query = "SELECT * " \
                f"FROM `{month}` " \
                f"WHERE pco_code = '{pco_code}' " \
                f"AND bnf_chemical_substance = '{bnf_chemical_substance}'"
    
    # Build temporary API call
    tmp_api_call  = f"{base_endpoint}" \
                    f"{action_method}" \
                    "resource_id=" \
                    f"{month}" \
                    "&" \
                    "sql=" \
                    f"{urllib.parse.quote(tmp_query)}" # Encode spaces in the url
    
    # Grab the response JSON as a temporary list
    tmp_response = requests.get(tmp_api_call).json()
    
    # Extract records in the response to a temporary dataframe
    tmp_df = pd.json_normalize(tmp_response['result']['result']['records'])
    
    # Bind the temporary data to the main dataframe
    for_loop_df = for_loop_df.append(tmp_df)

Lets have a quick look at the data

In [ ]:
for_loop_df.head()

## 5.2. Async

We can call the API asynchronously and this will result in an approx 10x speed 
increase over a for loop for large resource_names by vectorising our approach.

Construct the SQL query as a function

In [ ]:
def async_query(resource_name):
    query = "SELECT * " \
            f"FROM `{resource_name}` " \
            f"WHERE pco_code = '{pco_code}' " \
            f"AND bnf_chemical_substance = '{bnf_chemical_substance}'"
    return(query)

Create the API calls

In [ ]:
async_api_calls = []
for x in resource_name_list:
    async_api_calls.append(
        f"{base_endpoint}" \
        f"{action_method}" \
        "resource_id=" \
        f"{x}" \
        "&" \
        "sql=" \
        f"{urllib.parse.quote(async_query(x))}" # Encode spaces in the url 
    )

Use grequests to get the results

In [ ]:
dd = (grequests.get(u) for u in async_api_calls)
res = grequests.map(dd)

Check that everything is a success

In [ ]:
for x in res:
    if x.ok:
        print(True)
    else:
        print(False)

Parse the output into a list of dataframes and concatenate the results

In [ ]:
async_df = pd.DataFrame()

for x in res:
    # Grab the response JSON as a temporary list
    tmp_response = x.json()
    
    # Extract records in the response to a temporary dataframe
    tmp_df = pd.json_normalize(tmp_response['result']['result']['records'])
    
    # Bind the temporary data to the main dataframe
    async_df = async_df.append(tmp_df)

Lets have a quick look at the data

In [ ]:
async_df.head()

# 6. Export the data

In [ ]:
single_month_df.to_csv('single_month.csv')
for_loop_df.to_csv('for_loop.csv')
async_df.to_csv('async.csv')